# <font color="red">**This notebook is outdated and may no longer work properly**</font>

In [ ]:
import sys, os
from pathlib import Path
ROOT = Path.cwd().parent
if str(ROOT / 'src') not in sys.path:
    sys.path.insert(0, str(ROOT / 'src'))
print('Added to sys.path:', ROOT / 'src')


### Imports

In [ ]:
import os
import sys
import os
import torch
from IPython.display import Image, display

sys.path.insert(0, os.path.abspath("src"))

from config import *
from datasets.DataManager import DataManager
from AlexNet import AlexNet
from neural_data.AllenDataLoader import AllenDataLoader
from neural_data.AllenDataViewer import AllenDataViewer

In [ ]:
!bat ../src/config.py

In [ ]:
# This cell is used for constants override

DOWNLOAD_DATA = False
DOWNLOAD_ALLEN_DATA = False
CONVERT_ALLEN_DATA = False
VISUALIZE_ALLEN_DATA = True

LOAD_MODEL = True
TRAIN = True
TEST = True

NUM_EPOCHS = 300

### Initialize Data and Model

In [ ]:
# detect device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print("Optimizations applied: Mixed Precision, Optimized DataLoader, Larger Batch Size")

data_manager = DataManager(
    data_path=MINI_IMAGENET_PATH,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    train_split=TRAIN_SPLIT,
    val_split=VAL_SPLIT
)
data_manager.setup()

model_manager = AlexNet(
    data_manager=data_manager,
    num_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    dropout_rate=DROPOUT_RATE,
    patience=PATIENCE,
    label_smoothing=LABEL_SMOOTHING
)

#### Load Model

In [ ]:
if LOAD_MODEL:
    if os.path.exists(CHECKPOINT_PATH):
        try:
            model_manager.load_model(CHECKPOINT_PATH)
        except Exception as e:
            print(f"Could not warm-start from {CHECKPOINT_PATH}: {e}")
    else:
        print(f"No checkpoint found at {CHECKPOINT_PATH}")

#### Train

In [ ]:
if TRAIN:
    #? -------------- Training --------------
    # Train the model
    training_history = model_manager.train()

    # Plot training history
    model_manager.plot_training_history()
    #? ---------------------------------------


### Test

In [ ]:
if TEST:
    test_loss, test_accuracy, test_accuracy5 = model_manager.test()

    print(f"Test Results - Loss: {test_loss:.4f}, Acc-Top1: {test_accuracy:.2f}%, Acc-Top5: {test_accuracy5:.2f}%")

### Allen Dataset

In [ ]:
dataset = "neuropixels"   # or "calcium"
out = None                # output path for the PKL
path ="AllenData/neuropixels.zarr"               # path to an existing PKL

data_loader = AllenDataLoader(dataset=dataset)

if DOWNLOAD_ALLEN_DATA:
    saved_path = data_loader.download(out_path=out)

Convert data to a compatible type

In [ ]:
if CONVERT_ALLEN_DATA:
    os.system("REMOVE_ENV=1 src/neural_data/data_converter.sh")

Inspect data

In [ ]:
if VISUALIZE_ALLEN_DATA:
    data_viewer = AllenDataViewer(zarr_dir=str(ZARR_NEUROPIXELS))
    data_viewer.summary

Plot a grid of stimuli frames

In [ ]:
if VISUALIZE_ALLEN_DATA:
    stimuli_out = ROOT / Path("assets/stimuli_grid.png")
    stimuli_path = data_viewer.plot_stimuli_grid(
        out_path=str(stimuli_out),
        max_images=12,
        show=True
    )

Plot a heatmap of neural responses (optionally filtered by visual area)

In [ ]:
VISUAL_AREA = "VISp"

if VISUALIZE_ALLEN_DATA:
    responses_out = ROOT / Path("assets/neural_responses.png")
    responses_path = data_viewer.plot_neural_responses(
        out_path=str(responses_out),
        visual_area="VISrl",  # set e.g. "VISp" to filter if coord exists
        max_units=164,             # limit units for readability
        max_trials=100,           # limit trials for readability
        show=True
    )

In [ ]:

if VISUALIZE_ALLEN_DATA:
    reliability_out = ROOT / Path("assets/reliability.png")

    data_viewer.plot_neuropixels_reliability(
        out_path=str(reliability_out),
        visual_areas=None,
        fraction_of_max=True,
        time_step_ms=0.1,
        show=True
    )